# Extrahierung der Textstufen

In [1]:
import lxml
from lxml import etree
import pandas as pd

In [2]:
tree = etree.parse("C:/Users/jO/Desktop/DH_MA_Arbeit/coding/modelliert_output_pos_vwdazb_trf.xml")

In [3]:
ns = {"t":'http://www.tei-c.org/ns/1.0'}
xml = "{http://www.w3.org/XML/1998/namespace}"

In [8]:
poem = tree.xpath("//t:div[@type='poem']/descendant-or-self::text()", namespaces=ns)

In [9]:
witnesses = tree.xpath("//t:div[@type='witness']/descendant-or-self::text()", namespaces=ns)

In [15]:
# amount of poems in xml
amountpoems = len(tree.xpath("//t:div[@type='poem']", namespaces=ns))
print(amountpoems)

1


In [17]:
# amount of witnesses in xml
amountwitnesses = len(tree.xpath("//t:div[@type='witness']", namespaces=ns))
print(amountwitnesses)

7


In [18]:
# amount of published in xml
amountpublished = len(tree.xpath("//t:div[@type='published']", namespaces=ns))
print(amountpublished)

1


In [21]:
# alle xml:ids der witnesses
tree.xpath("//t:div[@type='witness']/@xml:id", namespaces=ns)

['nr.1', 'nr.2', 'nr.3', 'nr.4', 'nr.5', 'nr.6', 'nr.af']

In [27]:
witness1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']/descendant-or-self::*/text()", namespaces=ns)
print(witness1)

['\n                    ', '\n                        ', 'der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV \n                            schließen_/VERB\n                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT \n                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN \n                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN\n                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN \n                            in_/ADP der_/DET ', 'Brust_/NOUN ,_/PUNCT \n                            ', 'auf_/ADP die_/DET Finger_/NOUN', ' mit_/ADP der_/DET linken_/ADJ Hand_/NOUN \n                            3_/NUM mal_/ADV geteilt_/VERB', ' \n                            ', 'Westentasche_/NOUN ,_/PUNCT', ' ', '(_/PUNCT', ' ', 'oder_/CCONJ als_/SCONJ \n                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX \n                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ \n                 

### Extrahierung vom "originalen" Typoskript: 
- <add ignorieren
- <reg ignorieren
- <del einbeziehen
- <orig einbeziehen

In [53]:
wit1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']", namespaces=ns)
linegroups = tree.xpath("//t:lg", namespaces=ns)
lines = tree.xpath("//t:l", namespaces=ns)

for linegroups in wit1: 
    for lines in linegroups: 
        wit1typ = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']//t:l/text() | //t:div[@type='witness' and @xml:id='nr.1']//t:del/text() | //t:div[@type='witness' and @xml:id='nr.1']//t:orig/text()", namespaces=ns)
        wit1typ = "".join(wit1typ)
        wit1typ.replace("\n", "").replace("\r", "")
print("Originaltyposkript:", wit1typ)

Originaltyposkript: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET  
                            Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ 
                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX 
                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ 
                            Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON
                            nötig_/ADV haben_/VERB ,_/PUNCT die_/DET ihre_/DET 
                            Gedanken_/NOUN durch_/ADP die_/DET Feder_/NOUN 
                            sichtbar_/ADV zu_/PART mac

### Extrahierung von @change-Values Versionen: 
- <add  von @change="_pen" <hi>einbeziehen</hi>
- <reg von @change="_pen" einbeziehen
- <del einbeziehen, wenn es nicht @change="_pen" ist 
- <orig einbeziehen, wenn <reg nicht von @change="_pen" ist

In [5]:
# alle Versionen nach den verschiedenen @change-Values:
# mal nur aus dem ersten Textträger <div type='witness' @xml:id='nr.1'>

divwit1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']//t:l/descendant-or-self::*", namespaces=ns)
allattributevalues = set()

for child in divwit1:
    attribute_value = child.attrib.get('change')
    if attribute_value:
        if attribute_value not in allattributevalues:
            allattributevalues.add(attribute_value)
print(f"Unterschiedliche Revisionen: {allattributevalues}\n")
            
        
for attributevalue in allattributevalues:
    textname = f"{attributevalue}text"
    text = tree.xpath(f"""//t:div[@type='witness' and @xml:id='nr.1']//t:l/text() | 
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:add[@change='{attributevalue}']/text() | 
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:del[not(@change='{attributevalue}')]/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:reg[@change='{attributevalue}']/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)

    textname = "".join(text)
    textname.replace("\n", "").replace("\r", "")
    print(f"Typoskript mit Revisionen nach {attributevalue}:\n {textname} \n \n")


Unterschiedliche Revisionen: {'#typewriter', '#black_pen'}

Typoskript mit Revisionen nach #typewriter:
 der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET  
                            Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ 
                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX 
                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ 
                            Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON
                            nötig_/ADV haben_/VERB ,_/PUNCT  ihre_/DET 
                            Gedanken_/NOUN durch_/ADP

### Versuch: Extrahierung aller Textstufen nach vorkommenden @change Attributevalues aller Textzeugen

In [102]:
# funktioniert prinzipiell, aber l und lg werden nicht hinzugefügt, die div type=textname wird aber immer mehrfach erstellt.
# Annäherung an Extrahierung von allem (Original und Textstufen nach @change-values) mit Änderung des xml-Files
divs = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div[@type='witness']", namespaces=ns)

processed_texts = set()

#Funktion, die den extrahierten xpath-text weiterverarbeitet, ein tag erstellt und den text im neuen tag speichert
def addtextasdiv(text, div, textname): 
    if textname in processed_texts:
        print(f"{textname} wurde schon verarbeitet!")
        return
    else:        
        text = "".join(text)
        text = text.replace("\n", "").replace("\r", "")
        print(f"\n Neuer extrahierter Text:\n > Name: {textname}: \n > Text: {text}")
        
        print(f"\n Der Text von '{textname}' wird in xml geladen ...")
        
        linetag = etree.Element("l")
        print("1. linetag wurde erstellt")
        linetag.text = text # den extrahierten text als Inhalt des neuen divs definieren 
        print("2. text wurde dem linetag hinzugefügt")
        
        linegrouptag = etree.Element("lg") 
        print("3. linegrouptag wurde erstellt")
        linegrouptag.append(linetag)
        print("4. linetag wurde dem linegrouptag hinzugefügt")

        divtag = etree.Element("div")
        divtag.set("type", f"{textname}")  # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen      
        print("5. divtag wurde erstellt")
        
        divtag.append(linegrouptag)
        print("6. linegrouptag wurde zu divtag hinzugefügt")
        
        div.append(divtag) # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen
        print("7. divtag wurde dem div hinzugefügt")
        
        print(f" ... und '{textname}' wurde zum xml hinzugefügt!\n")
        processed_texts.add(textname)
        print(f"verarbeitete texte sind: {processed_texts}")



for div in divs: 
    divtype = div.attrib.get('type')
    divid = div.attrib.get(f'{xml}id')
    divname = f"{divtype}_{divid}"
    print(f"! Für folgendes <div> werden alle @change-Versionen extrahiert: '{divname}' !")
    
    #original herausnehmen:
    textname = f"{divname}_original"
    text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
    if text: 
        addtextasdiv(text, div, textname) #funktion zur verarbeitung und speicherung des textes

    allattributevalues = set() #set mit allen attributevalues pro div, damit jede @change einmal beachtet wird    

    lines = tree.xpath(f"//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/descendant-or-self::*", namespaces=ns) #versionen
    
    for line in lines:
        attribute_value = line.attrib.get('change') #unterschiedliche values von @change in allen tags innerhalb einer line abrufen
        #print(attribute_value)
        if attribute_value: 
            if attribute_value not in allattributevalues:
                allattributevalues.add(attribute_value) # unterschiedliche values von @change in das set hinzufügen
    print(f"\n Es gibt folgende Revisionen im Text '{divname}': {allattributevalues}\n") # printen, um sicher zu sein 
    
    for attributevalue in allattributevalues: # pro einzigartigem value von @change, das im set gespeichert wurde, wird jetzt der text abgerufen
        textname = f"{divname}_{attributevalue}"
        text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
        if text: 
            addtextasdiv(text, div, textname)


tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)


! Für folgendes <div> werden alle @change-Versionen extrahiert: 'witness_nr.1' !

 Neuer extrahierter Text:
 > Name: witness_nr.1_original: 
 > Text: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV                             schließen_/VERB                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT                             Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN                             als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN                             in_/ADP der_/DET                              Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ                             müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX                             dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ                             Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON                            nötig_/ADV haben_/VERB ,_/PUNCT die_/DET ihre_/DET             

# versuch, alles richtig abzurufen, aber in einzelnen Schritten

In [101]:
# versuch ohne die funktion, alles wird gleich direkt gemacht!
# Annäherung an Extrahierung von allem (Original und Textstufen nach @change-values) mit Änderung des xml-Files
divs = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div[@type='witness']", namespaces=ns)

processed_texts = set()


for div in divs: 
    divtype = div.attrib.get('type')
    divid = div.attrib.get(f'{xml}id')
    divname = f"{divtype}_{divid}"
    print(f"! Für folgendes <div> werden alle @change-Versionen extrahiert: '{divname}' !")
    
    #original herausnehmen:
    textname = f"{divname}_original"
    text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
    text = "".join(text)
    text.replace("\n", "").replace("\r", "")
    print(f"\n Neuer extrahierter Text wird geladen:\n > Name: {textname}: \n > Text: {text}")    
    linetag = etree.Element("l")
    print("1. linetag wurde erstellt")
    linetag.text = text # den extrahierten text als Inhalt des neuen divs definieren 
    print("2. text wurde dem linetag hinzugefügt")
    linegrouptag = etree.Element("lg") 
    print("3. linegrouptag wurde erstellt")
    linegrouptag.append(linetag)
    print("4. linetag wurde dem linegrouptag hinzugefügt")
    divtag = etree.Element("div")
    divtag.set("type", f"{textname}")  # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen      
    print("5. divtag wurde erstellt")
    divtag.append(linegrouptag)
    print("6. linegrouptag wurde zu divtag hinzugefügt")
    div.append(divtag) # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen
    print("7. divtag wurde dem div hinzugefügt")
    
    print(f" ... und '{textname}' wurde zum xml hinzugefügt!\n")
    processed_texts.add(textname)
    print(f"verarbeitete texte sind: {processed_texts}")

    allattributevalues = set() #set mit allen attributevalues pro div, damit jede @change einmal beachtet wird    

    lines = tree.xpath(f"//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/descendant-or-self::*", namespaces=ns) #versionen
    
    for line in lines:
        attribute_value = line.attrib.get('change') #unterschiedliche values von @change in allen tags innerhalb einer line abrufen
        #print(attribute_value)
        if attribute_value: 
            if attribute_value not in allattributevalues:
                allattributevalues.add(attribute_value) # unterschiedliche values von @change in das set hinzufügen
    print(f"\n Es gibt folgende Revisionen im Text '{divname}': {allattributevalues}\n") # printen, um sicher zu sein 
    
    for attributevalue in allattributevalues: # pro einzigartigem value von @change, das im set gespeichert wurde, wird jetzt der text abgerufen
        textname = f"{divname}_{attributevalue}"
        text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
        text = "".join(text)
        text.replace("\n", "").replace("\r", "")
        print(f"\n Neuer extrahierter Text wird geladen:\n > Name: {textname}: \n > Text: {text}")    
    linetag = etree.Element("l")
    print("1. linetag wurde erstellt")
    linetag.text = text # den extrahierten text als Inhalt des neuen divs definieren 
    print("2. text wurde dem linetag hinzugefügt")
    linegrouptag = etree.Element("lg") 
    print("3. linegrouptag wurde erstellt")
    linegrouptag.append(linetag)
    print("4. linetag wurde dem linegrouptag hinzugefügt")
    divtag = etree.Element("div")
    divtag.set("type", f"{textname}")  # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen      
    print("5. divtag wurde erstellt")
    divtag.append(linegrouptag)
    print("6. linegrouptag wurde zu divtag hinzugefügt")
    div.append(divtag) # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen
    print("7. divtag wurde dem div hinzugefügt")


tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)


! Für folgendes <div> werden alle @change-Versionen extrahiert: 'witness_nr.1' !

 Neuer extrahierter Text wird geladen:
 > Name: witness_nr.1_original: 
 > Text: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET  
                            Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ 
                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX 
                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ 
                            Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON
                            nötig_/ADV haben_/VERB ,_/PUNCT die_/DE

In [40]:
#nochmal neuer versuch, wo innerhalb der function erst später abgerufen wird. 

# funktioniert prinzipiell!!! original-Version wird 2mal eingefügt, sonst alles einmal!!!
# Annäherung an Extrahierung von allem (Original und Textstufen nach @change-values) mit Änderung des xml-Files

processed_divs = set()
processed_texts = set()

#Funktion, die den extrahierten xpath-text weiterverarbeitet, ein tag erstellt und den text im neuen tag speichert
def addtext(text, div, textname): 
    if textname not in processed_texts:    
        text = "".join(text)
        text = text.replace("\n", "").replace("\r", "")
        print(f"\n Neuer extrahierter Text:\n > Name: {textname}: \n > Text: {text}")
        print(f"\n Der Text von '{textname}' wird in xml geladen ...")
        linetag = etree.Element("l")
        print("1. linetag wurde erstellt")
        linetag.text = text # den extrahierten text als Inhalt des neuen divs definieren 
        print("2. text wurde dem linetag hinzugefügt")
        linegrouptag = etree.Element("lg") 
        print("3. linegrouptag wurde erstellt")
        linegrouptag.append(linetag)
        print("4. linetag wurde dem linegrouptag hinzugefügt")
        divtag = etree.Element("div")
        divtag.set("type", f"{textname}")  # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen      
        print("5. divtag wurde erstellt")
        divtag.append(linegrouptag)
        print("6. linegrouptag wurde zu divtag hinzugefügt")
        div.append(divtag) # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen
        print("7. divtag wurde dem div hinzugefügt")  
        print(f" ... und '{textname}' wurde zum xml hinzugefügt!\n")
        processed_texts.add(textname)
        print(f"verarbeitete texte sind: {processed_texts}")
    else:
        print(f"{textname} wurde schon verarbeitet!")
        return    
        
def processalldivs(divs):       
    for div in divs: 
        divtype = div.attrib.get('type')
        divid = div.attrib.get(f'{xml}id')
        divname = f"{divtype}_{divid}"
        print(f"! Für folgendes <div> werden alle @change-Versionen extrahiert: '{divname}' !")
        if divname not in processed_divs:
            #"original"-Version herausnehmen (=heißt: den Text in <l>, <del> und <orig> beachten):
            textname = f"{divname}_original"
            text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
            //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
            //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
            addtext(text, div, textname) #Funktion zur Verarbeitung und Vpeicherung des extrahierten Textes
        
            allattributevalues = set() #set für alle @change-attributevalues pro div   
            
            # der Text mit den Revisions-Tags befindet sich jeweils im Linetag <l>:
            lines = tree.xpath(f"//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/descendant-or-self::*", namespaces=ns)
            
            for line in lines:
                attribute_value = line.attrib.get('change') #unterschiedliche values von @change in allen tags innerhalb einer line abrufen
                if attribute_value: 
                    if attribute_value not in allattributevalues:
                        allattributevalues.add(attribute_value) # unterschiedliche values von @change in das set hinzufügen
            print(f"\n Es gibt folgende Revisionen im Text '{divname}': {allattributevalues}\n") # alle gefundenen @change-values printen 
            
            for attributevalue in allattributevalues: # pro @change-value, das im set gespeichert wurde, wird der spezifische Revisionstext abgerufen
                textname = f"{divname}_{attributevalue}"
                # pro Change Text beachten in <l>, <add>, <reg> aber nicht in <del> & <orig> (wenn reg dasselbe value hat wie <orig>)
                text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
                addtext(text, div, textname) #Funktion zur Verarbeitung und Speicherung des Textes
            processed_divs.add(divname)
        else:
            print("All done")
            return


In [58]:
#nochmal neuer versuch, wo innerhalb der function erst später abgerufen wird. 

# funktioniert prinzipiell!!! original-Version wird 2mal eingefügt, sonst alles einmal!!!
# Annäherung an Extrahierung von allem (Original und Textstufen nach @change-values) mit Änderung des xml-Files

processed_divs = set()
processed_texts = set()

#Funktion, die den extrahierten xpath-text weiterverarbeitet, ein tag erstellt und den text im neuen tag speichert
def addtext(text, div, textname): 
    if textname not in processed_texts:    
        text = "".join(text)
        text = text.replace("\n", "").replace("\r", "")
        print(f"\n Neuer extrahierter Text:\n > Name: {textname}: \n > Text: {text}")
        print(f"\n Der Text von '{textname}' wird in xml geladen ...")
        linetag = etree.Element("l")
        print("1. linetag wurde erstellt")
        linetag.text = text # den extrahierten text als Inhalt des neuen divs definieren 
        print("2. text wurde dem linetag hinzugefügt")
        linegrouptag = etree.Element("lg") 
        print("3. linegrouptag wurde erstellt")
        linegrouptag.append(linetag)
        print("4. linetag wurde dem linegrouptag hinzugefügt")
        divtag = etree.Element("div")
        divtag.set("type", f"{textname}")  # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen      
        print("5. divtag wurde erstellt")
        divtag.append(linegrouptag)
        print("6. linegrouptag wurde zu divtag hinzugefügt")
        div.append(divtag) # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen -> macht keinen Sinn!!
        print("7. divtag wurde dem div hinzugefügt")  
        print(f" ... und '{textname}' wurde zum xml hinzugefügt!\n")
        processed_texts.add(textname)
        if textname in processed_texts: 
            print(f"verarbeitete texte sind: {processed_texts}")
            return
    else:
        print(f"{textname} wurde schon verarbeitet!")
        return    
        
def processalldivs(divs):  
    divlen = len(divs)
    counter = 0
    print(f"Es sind {len(divs)} divs im XML-Dokument")
    if counter <= divlen:
        for div in divs: 
            divtype = div.attrib.get('type')
            divid = div.attrib.get(f'{xml}id')
            divname = f"{divtype}_{divid}"
            print(f"! Für folgendes <div> werden alle @change-Versionen extrahiert: '{divname}' !")
            if divname not in processed_divs:
                #"original"-Version herausnehmen (=heißt: den Text in <l>, <del> und <orig> beachten):
                textname = f"{divname}_original"
                text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
                addtext(text, div, textname) #Funktion zur Verarbeitung und Vpeicherung des extrahierten Textes
            
                allattributevalues = set() #set für alle @change-attributevalues pro div   
                
                # der Text mit den Revisions-Tags befindet sich jeweils im Linetag <l>:
                lines = tree.xpath(f"//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/descendant-or-self::*", namespaces=ns)
                
                for line in lines:
                    attribute_value = line.attrib.get('change') #unterschiedliche values von @change in allen tags innerhalb einer line abrufen
                    if attribute_value: 
                        if attribute_value not in allattributevalues:
                            allattributevalues.add(attribute_value) # unterschiedliche values von @change in das set hinzufügen
                print(f"\n Es gibt folgende Revisionen im Text '{divname}': {allattributevalues}\n") # alle gefundenen @change-values printen 
                
                for attributevalue in allattributevalues: # pro @change-value, das im set gespeichert wurde, wird der spezifische Revisionstext abgerufen
                    textname = f"{divname}_{attributevalue}"
                    # pro Change Text beachten in <l>, <add>, <reg> aber nicht in <del> & <orig> (wenn reg dasselbe value hat wie <orig>)
                    text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
                    addtext(text, div, textname) #Funktion zur Verarbeitung und Speicherung des Textes
                processed_divs.add(divname)
                counter += 1
                print(f"\n!!! {counter} div/s wurden verarbeitet!\n")
            else:
                print("All done")
                return


In [59]:
divs = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div[@type='witness']", namespaces=ns)
processalldivs(divs)
tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)

Es sind 7 divs im XML-Dokument
! Für folgendes <div> werden alle @change-Versionen extrahiert: 'witness_nr.1' !

 Neuer extrahierter Text:
 > Name: witness_nr.1_original: 
 > Text: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV                             schließen_/VERB                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT                             Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN                             als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN                             in_/ADP der_/DET                              Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ                             müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX                             dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ                             Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON                            nötig_/ADV haben_/VERB ,_/PUNCT 

In [61]:

#sehr alter versuch
lines = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']//t:div[@type='witness']//t:l", namespaces=ns) #versionen

divs = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div", namespaces=ns)

for div in divs: 
    divtype = div.attrib.get('type')
    divid = div.attrib.get(f'{xml}id')
    divname = f"{divtype}_{divid}"
    print(divname)
    
    if divtype == "witness": 
        allattributevalues = set() #set mit allen attributevalues pro div, damit jede @change einmal beachtet wird    
        for line in lines:
            # alle @change-Versionen extrahieren, wie es unterschiedliche change-Attribute-Values gibt (#typewriter, #blue_pen, #black_pen etc.) 
            attribute_value = line.attrib.get('change') #unterschiedliche values von @change abrufen
           # print(attribute_value)
            if attribute_value: 
                if attribute_value not in allattributevalues:
                    allattributevalues.add(attribute_value) # unterschiedliche values von @change in das set hinzufügen
        print(f"Unterschiedliche Revisionen in {divname}: {allattributevalues}\n") # printen, um sicher zu sein 
        
        for attributevalue in allattributevalues: # pro einzigartigem value von @change, das im set gespeichert wurde, wird jetzt der text abgerufen
            textname = f"{divname}_{attributevalue}"
            text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
            //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
            //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
            //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
            //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
            //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
            addtextasdiv(text, divname)
    

witness_nr.1
Unterschiedliche Revisionen in witness_nr.1: set()

witness_nr.2
Unterschiedliche Revisionen in witness_nr.2: set()

witness_nr.3
Unterschiedliche Revisionen in witness_nr.3: set()

witness_nr.4
Unterschiedliche Revisionen in witness_nr.4: set()

witness_nr.5
Unterschiedliche Revisionen in witness_nr.5: set()

witness_nr.6
Unterschiedliche Revisionen in witness_nr.6: set()

witness_nr.af
Unterschiedliche Revisionen in witness_nr.af: set()

published_None


In [ ]:
# alles speichern: 
tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)

In [ ]:
# Annäherung an Extrahierung von allem (Original und Textstufen nach @change-values) mit Änderung des xml-Files

#divs = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div/descendant-or-self::*", namespaces=ns)
#divs = tree.xpath("//t:div[@type='witness']", namespaces=ns) #original
#divwit = tree.xpath("//t:div[@type='witness']//t:l/descendant-or-self::*", namespaces=ns)

processed_divs = set()  # um die durchgegangenen divs zu speichern











In [29]:
# versuch, die richtigen attribute und values abzurufen: 

poem = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div", namespaces=ns)
#div = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']//t:div", namespaces=ns)

for div_el in poem: 

    divtype = div_el.attrib.get('type')
    divid = div_el.attrib.get(f'{xml}id')
    divname = f"{divtype}_{divid}"
    print(divname)
      
    allattributevalues = set() 
    attribute_value = div_el.attrib.get('change') 
    if attribute_value:  
        if attribute_value not in allattributevalues:
            allattributevalues.add(attribute_value) 
    print(f"Unterschiedliche Revisionen: {divname} {allattributevalues}\n")



witness_nr.1
Unterschiedliche Revisionen: witness_nr.1 set()

witness_nr.2
Unterschiedliche Revisionen: witness_nr.2 set()

witness_nr.3
Unterschiedliche Revisionen: witness_nr.3 set()

witness_nr.4
Unterschiedliche Revisionen: witness_nr.4 set()

witness_nr.5
Unterschiedliche Revisionen: witness_nr.5 set()

witness_nr.6
Unterschiedliche Revisionen: witness_nr.6 set()

witness_nr.af
Unterschiedliche Revisionen: witness_nr.af set()

published_None
Unterschiedliche Revisionen: published_None set()



In [ ]:
# alle Versionen nach den verschiedenen @change-Values:
# für das gesamte xml-Dokument:

poem = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/descendant-or-self::*", namespaces=ns)
div = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div[@type='witness']//t:l/descendant-or-self::*", namespaces=ns)

for div in poem: 

    divtype = div.attrib.get('type')
    divid = div.attrib.get('xml:id')
    divname = f"{divtype}_{divid}"
                         
    allattributevalues = set()

    #original herausnehmen:   
    for child in div: 
        text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']/t:l/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
       
        text = "".join(text)
        text.replace("\n", "").replace("\r", "")
        print("Originaltyposkript:", text)
        
        # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen:
        newtag = etree.Element("div")
        # für dieses div ein @type-Attribute mit dem Value bestehend aus: Name des Ursprungs-div und "original"
        newtag.set("type", f"{divname}_original")
        # den extrahierten text als Inhalt des neuen divs definieren
        newtag.text = text
        # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen 
        div.append(newtag)
        #ende original herausnehmen

    # alle so viele Versionen extrahieren, wie es unterschiedliche change-Attribute-Values gibt (#typewriter, #blue_pen, #black_pen etc.) 
    for child in div:
        attribute_value = child.attrib.get('change')
        if attribute_value:
            if attribute_value not in allattributevalues:
                allattributevalues.add(attribute_value)
    print(f"Unterschiedliche Revisionen: {divname} {allattributevalues}\n")
                
            
    for attributevalue in allattributevalues:
        textname = f"{attributevalue}text"
        text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
    
        text = "".join(text)
        text.replace("\n", "").replace("\r", "")
        print(f"Typoskript mit Revisionen nach {attributevalue}:\n {text} \n \n")

        # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen:
        newtag = etree.Element("div")
        # für dieses div ein @type-Attribute mit dem Value bestehend aus: Name des Ursprungs-div und des zugehörigen @change-Values
        newtag.set("type", f"{divname}_{attributevalue}")
        # den extrahierten text als Inhalt des neuen divs definieren
        newtag.text = text
        # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen 
        div.append(newtag)
        

tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)